<a href="https://colab.research.google.com/github/ZathanZ61/EstruturasDeDados1Teste/blob/main/TeoriaFilas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Atividade: Simulação de Triagem Hospitalar (Protocolo de Manchester)Este notebook Python implementa um sistema de gestão de filas para uma triagem de enfermagem. O objetivo é ordenar pacientes de forma eficiente, combinando o grau de urgência (Protocolo de Manchester) com a ordem de chegada (FIFO).Funcionamento e EstruturasO sistema utiliza as seguintes estruturas para garantir a ordem e o desempenho:Dicionário: Para manter filas separadas para cada especialidade médica.Fila de Prioridade (heapq): Essencial para ordenar os pacientes. A prioridade é dada pela cor de Manchester (Vermelho > Laranja > Amarelo > Azul) e, em caso de empate, pelo horário de chegada.Eficiência: As operações de adicionar paciente e chamar o próximo têm complexidade $O(\log n)$, garantindo que o sistema seja rápido mesmo com um alto volume de pacientes.RequisitosO código realiza os seguintes cálculos e operações:Cadastro de pacientes com cor de urgência e hora de chegada.Determinação da ordem de atendimento por especialidade.Cálculo do horário previsto de atendimento (considerando o médico ocupado por 20 minutos).Cálculo do tempo médio de espera dos pacientes ainda na fila.

In [1]:
import heapq

# --- 1. CONFIGURAÇÕES GERAIS E CONSTANTES ---

# Tempo de consulta padrão em minutos
TEMPO_CONSULTA = 20

# Mapeamento de Cores para Prioridades Numéricas (Regra de Manchester)
# Maior número = Maior prioridade
PRIORIDADES = {
    "vermelho": 4,
    "laranja": 3,
    "amarelo": 2,
    "azul": 1
}

# Lista das especialidades médicas que terão filas separadas
ESPECIALIDADES = [
    "Clínica Médica", "Pediatria", "Ortopedia",
    "Cardiologia", "Ginecologia/Obstetrícia", "Otorrinolaringologia"
]

# --- 2. FUNÇÕES DE UTILITÁRIOS DE TEMPO ---

def hora_para_min(hora_str):
    """Converte a hora no formato 'HH:MM' para o total de minutos desde 00:00."""
    try:
        H, M = map(int, hora_str.split(':'))
        return H * 60 + M
    except:
        # Retorna 0 se a string de hora for inválida (segurança básica)
        return 0

def min_para_hora(min_total):
    """Converte o total de minutos desde 00:00 para o formato 'HH:MM'."""
    h = min_total // 60
    m = min_total % 60
    return f"{h:02d}:{m:02d}"

# --- 3. CLASSE PRINCIPAL DA TRIAGEM ---

class SistemaTriagem:
    def __init__(self):
        # Filas: Dicionário onde a chave é a especialidade e o valor é a Fila de Prioridade (Heap)
        self.filas = {esp: [] for esp in ESPECIALIDADES}

        # Agenda: Dicionário para saber quando cada médico ficará livre (em minutos desde 00:00)
        self.medico_livre = {esp: 0 for esp in ESPECIALIDADES}

        # Contador para garantir que pacientes com o MESMO minuto de chegada
        # e MESMA prioridade sejam tratados em ordem FIFO.
        self.contador_chegada = 0

    def adicionar_paciente(self, nome, idade, cor, esp, hora_chegada_str):
        if esp not in self.filas:
            print(f"Erro: Especialidade '{esp}' não existe.")
            return

        urgencia_num = PRIORIDADES.get(cor.lower(), 0)
        chegada_min = hora_para_min(hora_chegada_str)

        # 1. Estrutura de Dados do Paciente
        paciente = {
            'nome': nome,
            'idade': idade,
            'cor': cor.lower(),
            'chegada_min': chegada_min
        }

        # 2. Chave de Prioridade (O CORAÇÃO DO ALGORITMO)
        # O heapq usa o primeiro elemento da tupla para prioridade (menor valor = maior prioridade)
        # Usamos o NEGATIVO da urgência: (-4) é menor que (-3), garantindo que VERMELHO vá antes.
        # O segundo elemento é o minuto de chegada: garante FIFO (primeiro a chegar tem menor minuto).
        prioridade_chave = (-urgencia_num, chegada_min, self.contador_chegada)
        self.contador_chegada += 1

        # 3. Inserir na fila usando o Heap (O(log n) de complexidade)
        heapq.heappush(self.filas[esp], (prioridade_chave, paciente))
        print(f"✅ {nome} adicionado. Fila: {esp} | Prioridade: {cor.upper()}")

    # --- FUNÇÃO AUXILIAR PARA CÁLCULOS (Simulação da Agenda) ---
    # Esta função percorre a fila na ordem correta, mas SEM remover ninguém.
    def _simular_agenda_especialidade(self, especialidade):
        """Calcula os horários previstos de atendimento para todos na fila (O(n_s))."""
        fila_atual = self.filas[especialidade]
        agenda_prevista = []

        # Criamos uma cópia do heap para simulação, para não estragar a fila original
        temp_heap = list(fila_atual)
        heapq.heapify(temp_heap)

        # O tempo de início da primeira consulta é o tempo que o médico está livre
        tempo_simulado_livre = self.medico_livre[especialidade]

        for _ in range(len(temp_heap)):
            _, paciente = heapq.heappop(temp_heap) # Pega o próximo na ordem de prioridade
            chegada_min = paciente['chegada_min']

            # Regra: Início = max(chegada_do_paciente, fim_da_consulta_anterior)
            inicio_min = max(chegada_min, tempo_simulado_livre)
            fim_min = inicio_min + TEMPO_CONSULTA

            # A espera real só acontece se o início for depois da chegada
            espera_min = max(0, inicio_min - chegada_min)

            # Atualiza o tempo livre para o próximo da simulação
            tempo_simulado_livre = fim_min

            agenda_prevista.append({
                'nome': paciente['nome'],
                'cor': paciente['cor'],
                'chegada_min': chegada_min,
                'inicio_min': inicio_min,
                'fim_min': fim_min,
                'espera_min': espera_min
            })

        return agenda_prevista

    # --- REQUISITOS FUNCIONAIS ---

    def proximo(self, especialidade):
        """Remove e retorna o próximo paciente e atualiza o tempo de ocupação do médico (O(log n))."""
        if not self.filas[especialidade]:
            print(f"\nFila de {especialidade} vazia.")
            return None

        # Remove o paciente de maior prioridade/FIFO (O(log n))
        _, paciente = heapq.heappop(self.filas[especialidade])

        # Atualiza o Relógio do Médico (Agenda)
        chegada_min = paciente['chegada_min']
        tempo_livre = self.medico_livre[especialidade]

        inicio_min = max(chegada_min, tempo_livre)
        fim_min = inicio_min + TEMPO_CONSULTA

        self.medico_livre[especialidade] = fim_min

        print(f"\n🔔 CHAMANDO PRÓXIMO: {especialidade}")
        print(f"   Paciente: {paciente['nome']} ({paciente['cor'].upper()})")
        print(f"   Início: {min_para_hora(inicio_min)} | Fim: {min_para_hora(fim_min)}")
        return paciente

    def previsao_atendimento(self, especialidade):
        """Lista ordenada com horários previstos (O(n_s))."""
        agenda = self._simular_agenda_especialidade(especialidade)
        if not agenda: return

        print(f"\n--- Previsão de Atendimento: {especialidade} ---")
        print(f"Médico disponível a partir de: {min_para_hora(self.medico_livre[especialidade])}")
        for item in agenda:
            print(f"   {item['nome']} ({item['cor'].upper()}): Início {min_para_hora(item['inicio_min'])} | Fim {min_para_hora(item['fim_min'])}")

    def tempo_medio_espera(self, especialidade):
        """Calcula o tempo médio de espera (O(n_s))."""
        agenda = self._simular_agenda_especialidade(especialidade)
        if not agenda: return 0

        total_pacientes = len(agenda)
        # Soma a espera real prevista (inicio - chegada)
        total_espera_min = sum(item['espera_min'] for item in agenda)
        tempo_medio = total_espera_min / total_pacientes

        print(f"\n--- Tempo Médio de Espera: {especialidade} ---")
        print(f"   Média de espera (previsão): {tempo_medio:.2f} minutos.")
        return tempo_medio

    def listar_fila(self, especialidade):
        """Mostra pacientes na fila em ordem de atendimento (O(n_s log n_s) para exibição)."""
        temp_heap = list(self.filas[especialidade])
        temp_heap.sort(key=lambda x: x[0]) # Ordena pela chave de prioridade

        print(f"\n--- Ordem da Fila: {especialidade} (Total: {len(temp_heap)}) ---")
        for i, (chave, paciente) in enumerate(temp_heap):
            urgencia_reversa = -chave[0]
            cor = [c for c, p in PRIORIDADES.items() if p == urgencia_reversa][0].upper()

            print(f"   {i+1}. {paciente['nome']} ({cor}) | Chegada: {min_para_hora(paciente['chegada_min'])}")

# --- 4. EXECUÇÃO / TESTES ---

sistema = SistemaTriagem()

print("\n\n################ 1. Cadastro de Pacientes ################")

# Pacientes do Exemplo
sistema.adicionar_paciente("Maria Silva", 34, "Laranja", "Ortopedia", "08:05")  # (-3, 485)
sistema.adicionar_paciente("João Souza", 62, "Amarelo", "Clínica Médica", "08:07") # (-2, 487)
sistema.adicionar_paciente("Ana Lima", 5, "Vermelho", "Pediatria", "08:09")   # (-4, 489)
sistema.adicionar_paciente("Carla Dias", 41, "Azul", "Ortopedia", "08:10")    # (-1, 490)
sistema.adicionar_paciente("Pedro Neri", 58, "Laranja", "Cardiologia", "08:12") # (-3, 492)
sistema.adicionar_paciente("Rui Campos", 29, "Amarelo", "Ortopedia", "08:14") # (-2, 494)

# Teste de Estabilidade (FIFO para mesma prioridade) na Ortopedia:
# Lucas (-3, 495) deve ser chamado antes de Mariana (-3, 496)
sistema.adicionar_paciente("Lucas FIFO", 50, "Laranja", "Ortopedia", "08:15") # (-3, 495)
sistema.adicionar_paciente("Mariana FIFO", 30, "Laranja", "Ortopedia", "08:16") # (-3, 496)

print("\n\n################ 2. Análise Inicial das Filas ################")

sistema.listar_fila("Ortopedia")
sistema.previsao_atendimento("Ortopedia")
sistema.previsao_atendimento("Pediatria")

print("\n\n################ 3. Chamando Pacientes (Simulação) ################")

# 1ª Chamada Ortopedia (Maria)
sistema.proximo("Ortopedia")
# Fim Maria: 08:25 (Médico de Ortopedia livre às 08:25)

# 2ª Chamada Pediatria (Ana)
sistema.proximo("Pediatria")
# Fim Ana: 08:29 (Médico de Pediatria livre às 08:29)

# 3ª Chamada Ortopedia (Lucas, FIFO Laranja)
sistema.proximo("Ortopedia")
# Lucas chegou às 08:15, mas médico só livre às 08:25.
# Início: 08:25. Fim: 08:45

print("\n\n################ 4. Previsão e Métricas Atualizadas ################")

sistema.listar_fila("Ortopedia") # Rui e Carla na fila (Amarelo e Azul)
sistema.previsao_atendimento("Ortopedia") # Próximo (Mariana) inicia às 08:45
sistema.tempo_medio_espera("Ortopedia")

sistema.proximo("Clínica Médica")
sistema.proximo("Cardiologia")



################ 1. Cadastro de Pacientes ################
✅ Maria Silva adicionado. Fila: Ortopedia | Prioridade: LARANJA
✅ João Souza adicionado. Fila: Clínica Médica | Prioridade: AMARELO
✅ Ana Lima adicionado. Fila: Pediatria | Prioridade: VERMELHO
✅ Carla Dias adicionado. Fila: Ortopedia | Prioridade: AZUL
✅ Pedro Neri adicionado. Fila: Cardiologia | Prioridade: LARANJA
✅ Rui Campos adicionado. Fila: Ortopedia | Prioridade: AMARELO
✅ Lucas FIFO adicionado. Fila: Ortopedia | Prioridade: LARANJA
✅ Mariana FIFO adicionado. Fila: Ortopedia | Prioridade: LARANJA


################ 2. Análise Inicial das Filas ################

--- Ordem da Fila: Ortopedia (Total: 5) ---
   1. Maria Silva (LARANJA) | Chegada: 08:05
   2. Lucas FIFO (LARANJA) | Chegada: 08:15
   3. Mariana FIFO (LARANJA) | Chegada: 08:16
   4. Rui Campos (AMARELO) | Chegada: 08:14
   5. Carla Dias (AZUL) | Chegada: 08:10

--- Previsão de Atendimento: Ortopedia ---
Médico disponível a partir de: 00:00
   Maria Silva (LA

{'nome': 'Pedro Neri', 'idade': 58, 'cor': 'laranja', 'chegada_min': 492}